# Food Delivery Time Prediction Dataset

Este conjunto de datos está diseñado para **predecir los tiempos de entrega de comida** en función de diversos factores como la distancia, el clima, las condiciones del tráfico y la hora del día.
Ofrece un desafío práctico e interesante para **profesionales del aprendizaje automático**, especialmente aquellos interesados en **logística e investigación operativa**.

## Diccionario de Datos

| **Variable**               | **Descripción**                                                                                    |
| -------------------------- | -------------------------------------------------------------------------------------------------- |
| **Order_ID**               | Identificador único para cada pedido.                                                              |
| **Distance_km**            | Distancia de entrega en kilómetros.                                                                |
| **Weather**                | Condiciones climáticas durante la entrega (Clear, Rainy, Snowy, Foggy, Windy).                     |
| **Traffic_Level**          | Condiciones de tráfico clasificadas como **Low**, **Medium** o **High**.                           |
| **Time_of_Day**            | Momento del día en que se realizó la entrega (**Morning**, **Afternoon**, **Evening**, **Night**). |
| **Vehicle_Type**           | Tipo de vehículo utilizado para la entrega (**Bike**, **Scooter**, **Car**).                       |
| **Preparation_Time_min**   | Tiempo requerido para preparar el pedido, medido en minutos.                                       |
| **Courier_Experience_yrs** | Experiencia del repartidor en años.                                                                |
| **Delivery_Time_min**      | **Variable objetivo** — tiempo total de entrega en minutos.                                        |

## Importar librerias

In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.tree import DecisionTreeRegressor
from xgboost import XGBRegressor
import warnings
warnings.filterwarnings('ignore')

# Configuración de visualización
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 8)

## 1. Carga y Exploración de Datos

In [18]:
# Cargar el dataset
df = pd.read_csv('Food_delivery_times.csv')

# Información general
print(f"Dimensiones del DataFrame: {df.shape}")

print("\nPrimeras 5 filas:")
display(df.head())

print("\nResumen estadístico:")
display(df.describe())

print("\nTipos de datos:")
print(df.dtypes)

print("\nValores nulos por columna:")
print(df.isnull().sum())

Dimensiones del DataFrame: (1000, 9)

Primeras 5 filas:


,Order_ID,Distance_km,Weather,Traffic_Level,Time_of_Day,Vehicle_Type,Preparation_Time_min,Courier_Experience_yrs,Delivery_Time_min
0,522,7.93,Windy,Low,Afternoon,Scooter,12,1.0,43
1,738,16.42,Clear,Medium,Evening,Bike,20,2.0,84
2,741,9.52,Foggy,Low,Night,Scooter,28,1.0,59
3,661,7.44,Rainy,Medium,Afternoon,Scooter,5,1.0,37
4,412,19.03,Clear,Low,Morning,Bike,16,5.0,68



Resumen estadístico:


,Order_ID,Distance_km,Preparation_Time_min,Courier_Experience_yrs,Delivery_Time_min
count,1000.000000,1000.000000,1000.000000,970.000000,1000.000000
mean,500.500000,10.059970,16.982000,4.579381,56.732000
std,288.819436,5.696656,7.204553,2.914394,22.070915
min,1.000000,0.590000,5.000000,0.000000,8.000000
25%,250.750000,5.105000,11.000000,2.000000,41.000000
50%,500.500000,10.190000,17.000000,5.000000,55.500000
75%,750.250000,15.017500,23.000000,7.000000,71.000000
max,1000.000000,19.990000,29.000000,9.000000,153.000000



Tipos de datos:
Order_ID                    int64
Distance_km               float64
Weather                    object
Traffic_Level              object
Time_of_Day                object
Vehicle_Type               object
Preparation_Time_min        int64
Courier_Experience_yrs    float64
Delivery_Time_min           int64
dtype: object

Valores nulos por columna:
Order_ID                   0
Distance_km                0
Weather                   30
Traffic_Level             30
Time_of_Day               30
Vehicle_Type               0
Preparation_Time_min       0
Courier_Experience_yrs    30
Delivery_Time_min          0
dtype: int64


# HASTA AQUI 

In [ ]:

# %% [markdown]
# ## 3. Análisis Exploratorio de Datos (EDA)

# %%
# Distribución de la variable objetivo
plt.figure(figsize=(15, 5))

plt.subplot(1, 3, 1)
sns.histplot(df['Delivery_Time_min'], kde=True, bins=30)
plt.title('Distribución del Tiempo de Entrega')
plt.xlabel('Tiempo de Entrega (min)')
plt.ylabel('Frecuencia')

plt.subplot(1, 3, 2)
sns.boxplot(y=df['Delivery_Time_min'])
plt.title('Boxplot del Tiempo de Entrega')
plt.ylabel('Tiempo de Entrega (min)')

plt.subplot(1, 3, 3)
sns.scatterplot(x=df['Distance_km'], y=df['Delivery_Time_min'])
plt.title('Distancia vs Tiempo de Entrega')
plt.xlabel('Distancia (km)')
plt.ylabel('Tiempo de Entrega (min)')

plt.tight_layout()
plt.show()

# %%
# Análisis de variables categóricas
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# Weather vs Delivery Time
sns.boxplot(data=df, x='Weather', y='Delivery_Time_min', ax=axes[0, 0])
axes[0, 0].set_title('Tiempo de Entrega por Condición Climática')
axes[0, 0].tick_params(axis='x', rotation=45)

# Traffic Level vs Delivery Time
sns.boxplot(data=df, x='Traffic_Level', y='Delivery_Time_min', ax=axes[0, 1])
axes[0, 1].set_title('Tiempo de Entrega por Nivel de Tráfico')

# Time of Day vs Delivery Time
sns.boxplot(data=df, x='Time_of_Day', y='Delivery_Time_min', ax=axes[1, 0])
axes[1, 0].set_title('Tiempo de Entrega por Hora del Día')

# Vehicle Type vs Delivery Time
sns.boxplot(data=df, x='Vehicle_Type', y='Delivery_Time_min', ax=axes[1, 1])
axes[1, 1].set_title('Tiempo de Entrega por Tipo de Vehículo')

plt.tight_layout()
plt.show()

# %%
# Matriz de correlación para variables numéricas
numeric_cols = ['Distance_km', 'Preparation_Time_min', 'Courier_Experience_yrs', 'Delivery_Time_min']
correlation_matrix = df[numeric_cols].corr()

plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0, square=True)
plt.title('Matriz de Correlación - Variables Numéricas')
plt.show()

# %%
# Relación entre experiencia del repartidor y tiempo de entrega
plt.figure(figsize=(10, 6))
sns.scatterplot(data=df, x='Courier_Experience_yrs', y='Delivery_Time_min', alpha=0.6)
plt.title('Experiencia del Repartidor vs Tiempo de Entrega')
plt.xlabel('Experiencia (años)')
plt.ylabel('Tiempo de Entrega (min)')
plt.show()

# %% [markdown]
# ## 4. Preprocesamiento de Datos

# %%
# Copiar el dataset para preprocesamiento
df_processed = df.copy()

# Eliminar Order_ID ya que no es útil para el modelo
df_processed = df_processed.drop('Order_ID', axis=1)
print("Order_ID eliminado del dataset")

# %%
# Codificación de variables categóricas
label_encoders = {}

categorical_columns = ['Weather', 'Traffic_Level', 'Time_of_Day', 'Vehicle_Type']

print("=== CODIFICACIÓN DE VARIABLES CATEGÓRICAS ===")
for col in categorical_columns:
    le = LabelEncoder()
    df_processed[col] = le.fit_transform(df_processed[col])
    label_encoders[col] = le
    print(f"{col}: {dict(zip(le.classes_, le.transform(le.classes_)))}")

# %%
# Separar características y variable objetivo
X = df_processed.drop('Delivery_Time_min', axis=1)
y = df_processed['Delivery_Time_min']

print("Características (X):", X.shape)
print("Variable objetivo (y):", y.shape)

# %%
# Dividir en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print(f"Conjunto de entrenamiento: {X_train.shape}")
print(f"Conjunto de prueba: {X_test.shape}")

# %%
# Escalar características numéricas
scaler = StandardScaler()
numeric_features = ['Distance_km', 'Preparation_Time_min', 'Courier_Experience_yrs']

X_train_scaled = X_train.copy()
X_test_scaled = X_test.copy()

X_train_scaled[numeric_features] = scaler.fit_transform(X_train[numeric_features])
X_test_scaled[numeric_features] = scaler.transform(X_test[numeric_features])

print("Características numéricas escaladas exitosamente")

# %% [markdown]
# ## 5. Modelado y Entrenamiento

# %%
# Definir modelos a probar
models = {
    'Linear Regression': LinearRegression(),
    'Ridge Regression': Ridge(),
    'Lasso Regression': Lasso(),
    'Decision Tree': DecisionTreeRegressor(random_state=42),
    'Random Forest': RandomForestRegressor(random_state=42),
    'Gradient Boosting': GradientBoostingRegressor(random_state=42),
    'XGBoost': XGBRegressor(random_state=42)
}

print("=== ENTRENAMIENTO DE MODELOS ===")

# %%
# Entrenar y evaluar modelos
results = {}

for name, model in models.items():
    print(f"Entrenando {name}...")
    
    # Entrenar modelo
    if name in ['Linear Regression', 'Ridge Regression', 'Lasso Regression']:
        model.fit(X_train_scaled, y_train)
        y_pred = model.predict(X_test_scaled)
    else:
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
    
    # Calcular métricas
    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test, y_pred)
    
    results[name] = {
        'MAE': mae,
        'MSE': mse,
        'RMSE': rmse,
        'R2': r2
    }
    
    print(f"{name}:")
    print(f"  MAE: {mae:.2f}")
    print(f"  RMSE: {rmse:.2f}")
    print(f"  R²: {r2:.4f}")
    print("-" * 40)

# %%
# Comparación de modelos
results_df = pd.DataFrame(results).T
results_df = results_df.sort_values('RMSE')

plt.figure(figsize=(12, 6))
bars = plt.bar(results_df.index, results_df['RMSE'], color='skyblue')
plt.title('Comparación de Modelos - RMSE (Menor es mejor)')
plt.xlabel('Modelos')
plt.ylabel('RMSE')
plt.xticks(rotation=45)
plt.grid(axis='y', alpha=0.3)

# Añadir valores en las barras
for bar in bars:
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2., height + 0.1,
             f'{height:.2f}', ha='center', va='bottom')

plt.tight_layout()
plt.show()

# %% [markdown]
# ## 6. Optimización del Mejor Modelo

# %%
# Seleccionar el mejor modelo basado en RMSE
best_model_name = results_df.index[0]
print(f"Mejor modelo: {best_model_name}")

# %%
# Optimización de hiperparámetros para Random Forest (ejemplo)
if best_model_name == 'Random Forest':
    print("Optimizando Random Forest...")
    param_grid = {
        'n_estimators': [100, 200, 300],
        'max_depth': [10, 20, None],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4]
    }
    
    rf = RandomForestRegressor(random_state=42)
    grid_search = GridSearchCV(rf, param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
    grid_search.fit(X_train, y_train)
    
    best_rf = grid_search.best_estimator_
    print(f"Mejores parámetros: {grid_search.best_params_}")
    
    # Evaluar el modelo optimizado
    y_pred_best = best_rf.predict(X_test)
    
    mae_best = mean_absolute_error(y_test, y_pred_best)
    rmse_best = np.sqrt(mean_squared_error(y_test, y_pred_best))
    r2_best = r2_score(y_test, y_pred_best)
    
    print(f"Modelo Optimizado - MAE: {mae_best:.2f}, RMSE: {rmse_best:.2f}, R²: {r2_best:.4f}")

# %% [markdown]
# ## 7. Análisis de Importancia de Características

# %%
# Usar el mejor modelo para análisis de importancia
if best_model_name == 'Random Forest':
    best_model = RandomForestRegressor(random_state=42)
    best_model.fit(X_train, y_train)
elif best_model_name == 'XGBoost':
    best_model = XGBRegressor(random_state=42)
    best_model.fit(X_train, y_train)
else:
    best_model = models[best_model_name]

print(f"Modelo {best_model_name} entrenado para análisis de importancia")

# %%
# Importancia de características
if hasattr(best_model, 'feature_importances_'):
    feature_importance = pd.DataFrame({
        'feature': X.columns,
        'importance': best_model.feature_importances_
    }).sort_values('importance', ascending=False)
    
    plt.figure(figsize=(10, 6))
    sns.barplot(data=feature_importance, x='importance', y='feature')
    plt.title(f'Importancia de Características - {best_model_name}')
    plt.xlabel('Importancia')
    plt.tight_layout()
    plt.show()
    
    print("Importancia de características:")
    display(feature_importance)

# %% [markdown]
# ## 8. Predicciones y Evaluación Final

# %%
# Hacer predicciones con el mejor modelo
if best_model_name in ['Linear Regression', 'Ridge Regression', 'Lasso Regression']:
    final_predictions = best_model.predict(X_test_scaled)
else:
    final_predictions = best_model.predict(X_test)

print("Predicciones generadas con el mejor modelo")

# %%
# Visualizar predicciones vs valores reales
plt.figure(figsize=(10, 6))
plt.scatter(y_test, final_predictions, alpha=0.6)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
plt.xlabel('Valores Reales')
plt.ylabel('Predicciones')
plt.title('Predicciones vs Valores Reales')
plt.show()

# %%
# Residual plot
residuals = y_test - final_predictions

plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.scatter(final_predictions, residuals, alpha=0.6)
plt.axhline(y=0, color='r', linestyle='--')
plt.xlabel('Predicciones')
plt.ylabel('Residuales')
plt.title('Gráfico de Residuales')

plt.subplot(1, 2, 2)
sns.histplot(residuals, kde=True, bins=30)
plt.xlabel('Residuales')
plt.ylabel('Frecuencia')
plt.title('Distribución de Residuales')

plt.tight_layout()
plt.show()

# %%
# Métricas finales
final_mae = mean_absolute_error(y_test, final_predictions)
final_rmse = np.sqrt(mean_squared_error(y_test, final_predictions))
final_r2 = r2_score(y_test, final_predictions)

print("=== MÉTRICAS FINALES DEL MODELO ===")
print(f"MAE: {final_mae:.2f} minutos")
print(f"RMSE: {final_rmse:.2f} minutos")
print(f"R²: {final_r2:.4f}")

# %%
# Error porcentual promedio
percentage_error = np.mean(np.abs((y_test - final_predictions) / y_test)) * 100
print(f"Error porcentual promedio: {percentage_error:.2f}%")

# %% [markdown]
# ## 9. Predicción en Nuevos Datos

# %%
# Función para hacer predicciones en nuevos datos
def predict_delivery_time(distance, weather, traffic, time_of_day, 
                         vehicle_type, prep_time, courier_exp, model=best_model, scaler=scaler):
    
    # Codificar variables categóricas
    weather_encoded = label_encoders['Weather'].transform([weather])[0]
    traffic_encoded = label_encoders['Traffic_Level'].transform([traffic])[0]
    time_encoded = label_encoders['Time_of_Day'].transform([time_of_day])[0]
    vehicle_encoded = label_encoders['Vehicle_Type'].transform([vehicle_type])[0]
    
    # Crear array de características
    features = np.array([[distance, weather_encoded, traffic_encoded, time_encoded, 
                         vehicle_encoded, prep_time, courier_exp]])
    
    # Hacer predicción
    if best_model_name in ['Linear Regression', 'Ridge Regression', 'Lasso Regression']:
        # Escalar características numéricas
        features_scaled = features.copy()
        features_scaled[:, [0, 5, 6]] = scaler.transform(features[:, [0, 5, 6]])
        prediction = model.predict(features_scaled)[0]
    else:
        prediction = model.predict(features)[0]
    
    return max(prediction, 15)  # Mínimo 15 minutos

# %%
# Ejemplo de predicción
print("=== EJEMPLO DE PREDICCIÓN ===")
example_prediction = predict_delivery_time(
    distance=5.2,
    weather='Rainy',
    traffic='Medium',
    time_of_day='Evening',
    vehicle_type='Bike',
    prep_time=25,
    courier_exp=3.5
)

print(f"Tiempo de entrega predicho: {example_prediction:.1f} minutos")

# %%
# Múltiples ejemplos de predicción
print("\n=== PREDICCIONES MÚLTIPLES ===")
examples = [
    (3.0, 'Clear', 'Low', 'Afternoon', 'Scooter', 15, 5.0),
    (8.5, 'Rainy', 'High', 'Evening', 'Car', 30, 2.0),
    (12.0, 'Clear', 'Medium', 'Morning', 'Bike', 20, 7.5)
]

for i, (dist, weather, traffic, time, vehicle, prep, exp) in enumerate(examples, 1):
    pred = predict_delivery_time(dist, weather, traffic, time, vehicle, prep, exp)
    print(f"Ejemplo {i}: {pred:.1f} minutos")

# %% [markdown]
# ## 10. Conclusiones y Recomendaciones

# %%
print("=== CONCLUSIONES ===")
print(f"1. El mejor modelo es: {best_model_name}")
print(f"2. Precisión del modelo (R²): {final_r2:.4f}")
print(f"3. Error promedio: {final_mae:.2f} minutos")
print(f"4. Las características más importantes son: {list(feature_importance['feature'].head(3)) if 'feature_importance' in locals() else 'N/A'}")

print("\n=== RECOMENDACIONES ===")
print("1. Considerar la distancia y tiempo de preparación como factores clave")
print("2. Las condiciones climáticas y de tráfico afectan significativamente el tiempo de entrega")
print("3. La experiencia del repartidor ayuda a reducir el tiempo de entrega")
print("4. Utilizar este modelo para optimizar rutas y asignación de repartidores")
print("5. Implementar el modelo en producción para predicciones en tiempo real")

# %%
# Guardar el modelo entrenado (opcional)
import joblib

model_data = {
    'model': best_model,
    'scaler': scaler,
    'label_encoders': label_encoders,
    'feature_names': X.columns.tolist()
}

joblib.dump(model_data, 'food_delivery_model.pkl')
print("Modelo guardado como 'food_delivery_model.pkl'")